In [ ]:
import pandas as pd
from django.core.exceptions import ObjectDoesNotExist
from django.db.models.functions import Length

In [ ]:
file = "../rita_data_ibk/generated_files/adm_action_type_occurences.csv"
df = pd.read_csv(file)

In [ ]:
adm_col, _ = SkosCollection.objects.get_or_create(name='adm-action-type')
other, _ = SkosConcept.objects.get_or_create(pref_label='Sonstiges', pref_label_lang='ger')
other.collection.add(adm_col)

In [ ]:
for i, row in df.iterrows():
    if row['name'] > 4 and row[0]:
        con, _ = SkosConcept.objects.get_or_create(
            pref_label=row[0],
            pref_label_lang='ger'
        )
        con.collection.add(adm_col)

In [ ]:
file = "../rita_data_ibk/generated_files/out.csv"

In [ ]:
df = pd.read_csv(file)

In [ ]:
pl, _ = Place.objects.get_or_create(
    name='Bolzano',
    geonames_id="http://www.geonames.org/3181913/bolzano.html",
    lat = 46.49067,
    lng = 11.33982
)

In [ ]:
archiv, _ = Institution.objects.get_or_create(
    written_name="Südtiroler Landesarchiv",
    abbreviation="SLA"
)
archiv.location = pl
archiv.save()

In [ ]:
for i, row in df.iterrows():
    if len(row['entry_signatur']) > 250:
        pass
    else:
        vb, _ = VerfachBuch.objects.get_or_create(
            signatur=row['vb_signatur'],
            year = "{}-01-01".format(row['year'])
        )
        vb.repo = archiv
        vb.save()
        entry, _ = VfbEntry.objects.get_or_create(
            entry_signatur=row['entry_signatur']
        )
        entry.located_in = vb
        entry.vollregest = row['text']
        entry.inventory = row['inv']
        try:
            adm_type = SkosConcept.objects.get(pref_label=row['first_token'])
        except ObjectDoesNotExist:
            adm_type, _ = SkosConcept.objects.get_or_create(
                broader_concept=other,
                pref_label=row['first_token'],
                pref_label_lang='ger'
            )
            adm_type.collection.add(adm_col)
        entry.adm_action_type = adm_type
        entry.save()

In [ ]:
from annotations.utils import create_ner_sample_from_qs
# spacyurl = "https://spacyapp.acdh-dev.oeaw.ac.at/query/nerparser-api/"
spacyurl = "http://127.0.0.1:7000/query/nerparser-api/"

In [ ]:
for x in create_ner_sample_from_qs(
    'summaries', 'vfbentry', 'vollregest', spacyurl
):
    pass

In [ ]:
for x in NerSample.objects.all():
    ners = x.return_ents()
    if len(ners) == 0:
        pass
    else:
        for ent in ners:
            try:
                pers = Person.objects.create(written_name=ent)
                x.entry.mentioned_person.add(pers)
            except Exception as e:
                print("Error for ent: {} in VfbEntry-ID: {}. Error: {}".format(ent, x.id, e))

In [ ]:
Person.objects.annotate(text_len=Length('written_name')).filter(text_len__lt=9).delete()

In [ ]:
for x in Person.objects.all():
    names = x.written_name.split()
    if len(names) > 1:
        x.forename = names[0]
        if names[1].startswith('['):
            try:
                x.name = names[2]
            except IndexError:
                print(names)
        else:
            x.name = names[1]
        x.save()
    else:
        pass

In [ ]:
NerSample.objects.all().delete()

In [ ]:
# was a run to detect professions
for x in create_ner_sample_from_qs(
    'entities', 'person', 'written_name', spacyurl
):
    pass

In [ ]:
# derive a list of professions
jobs = []
for x in NerSample.objects.all():
    ners = x.return_ents()
    if len(ners) == 0:
        pass
    else:
        for ent in ners:
            if len(ent) > 3:
                jobs.append(ent)

In [ ]:
prof_coll, _ = SkosCollection.objects.get_or_create(name='profession')

In [ ]:
# add professions
for x in Person.objects.all():
    for y in set(jobs):
        if y in x.written_name:
            job, _ = SkosConcept.objects.get_or_create(
                pref_label=y
            )
            job.collection.add(prof_coll)
            x.profession.add(job)

In [ ]:
Ner.objects.all().delete()